# Queueing Simulation

In [1]:
# Dependencies
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# R packages via python
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector


In [3]:
pandas2ri.activate()
importr('queuecomputer')

rpy2.robjects.packages.Package as a <module 'queuecomputer'>

In [4]:
# Constants
lamdba = 1             # average number of arrivals per time period
walkin_mean = 2        # average time interval between walk-ins in minutes
walkin_sd = 0.5        # standard deviation of walk-ins in minutes
service_time_mean = 3  # time in minutes to complete service and allow next object in queue
service_time_sd = 0.5  # standard deviation of service time

In [5]:
# Model Parameters
number_of_people = 25


In [6]:
# calc arrival time and service time interval
cumulative_time = np.random.normal(loc=walkin_mean, scale=walkin_sd, size=number_of_people).cumsum()
# cumulative_time = np.insert(cumulative_time, 0,0)
service_time = np.random.normal(loc=service_time_mean, scale=service_time_sd, size=number_of_people)
# service_time = np.insert(service_time, 0,0)

In [22]:
# cumulative_time = np.random.exponential(1,400).cumsum()
# service_time = np.random.normal(1,0.2, 400)

In [7]:
arrival = FloatVector(cumulative_time)
service = FloatVector(service_time)

In [8]:
simulation = r['queue_step'](arrivals=arrival, service=service,servers=1)

In [25]:
## List of 7 
##  $ departures     : num [1:100] 2.45 2.93 3.7 5.4 7.07 ...
##  $ server         : int [1:100] 1 1 1 1 1 1 1 1 1 1 ...
##  $ departures_df  : tibble [100 x 6] (S3: tbl_df/tbl/data.frame)
##  $ queuelength_df :'data.frame': 201 obs. of  2 variables:
##  $ systemlength_df:'data.frame': 201 obs. of  2 variables:
##  $ servers_input  : num 1
##  $ state          : num 101
##  - attr(*, "class")= chr [1:2] "queue_list" "list"

In [9]:
print(simulation[0])

[ 4.1534689   7.2442908  10.62736786 14.14399946 17.2376767  20.18949443
 23.2268702  25.52490583 28.02844522 31.16287971 33.69264001 36.71786918
 39.42888967 42.3134775  45.96839247 48.54311437 51.5221439  54.70701381
 57.66354151 61.21050105 64.69786804 67.17764969 70.31248864 73.25412676
 76.83553676]


In [10]:
print(simulation[1])

 [1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1



In [11]:
print(simulation[2])

# A tibble: 25 x 6
   arrivals service departures  waiting system_time server
      <dbl>   <dbl>      <dbl>    <dbl>       <dbl>  <int>
 1     1.41    2.74       4.15 4.44e-16        2.74      1
 2     3.86    3.09       7.24 2.89e- 1        3.38      1
 3     6.80    3.38      10.6  4.49e- 1        3.83      1
 4     9.20    3.52      14.1  1.43e+ 0        4.94      1
 5    11.3     3.09      17.2  2.86e+ 0        5.95      1
 6    13.7     2.95      20.2  3.55e+ 0        6.50      1
 7    16.2     3.04      23.2  3.95e+ 0        6.98      1
 8    17.8     2.30      25.5  5.41e+ 0        7.71      1
 9    20.4     2.50      28.0  5.11e+ 0        7.61      1
10    22.6     3.13      31.2  5.39e+ 0        8.52      1
# ... with 15 more rows



In [15]:
print(simulation[3])

       times queuelength
1   0.000000           0
2   1.410679           1
3   1.410679           0
4   3.864763           1
5   4.153469           0
6   6.795529           1
7   7.244291           0
8   9.200562           1
9  10.627368           0
10 11.283835           1
11 13.688997           2
12 14.143999           1
13 16.242993           2
14 17.237677           1
15 17.818087           2
16 20.189494           1
17 20.417398           2
18 22.639483           3
19 23.226870           2
20 25.441872           3
21 25.524906           2
22 27.135080           3
23 28.028445           2
24 29.013158           3
25 31.026922           4
26 31.162880           3
27 33.692640           2
28 33.726752           3
29 35.234766           4
30 36.717869           3
31 37.388891           4
32 39.207393           5
33 39.428890           4
34 41.252310           5
35 42.313478           4
36 42.798143           5
37 44.285450           6
38 45.968392           5
39 46.448252           6
